In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
path_train = '/content/gdrive/MyDrive/ai-playground/images/spectrogram/dataset/train'
path_test = '/content/gdrive/MyDrive/ai-playground/images/spectrogram/dataset/test'

In [6]:
x_train=[]

for folder in os.listdir(path_train):
    if folder != '.DS_Store':
        sub_path=path_train+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)
        try:
          img_arr=cv2.resize(img_arr,(224,224))
        except:
          break
        x_train.append(img_arr)

x_test=[]

for folder in os.listdir(path_test):
    if folder != '.DS_Store':
        sub_path=path_test+"/"+folder

    for img in os.listdir(sub_path):
        if img.endswith("png"):

          image_path=sub_path+"/"+img

          img_arr=cv2.imread(image_path)

          img_arr=cv2.resize(img_arr,(224,224))

          x_test.append(img_arr)

In [7]:
train_x=np.array(x_train)
test_x=np.array(x_test)

In [8]:
train_x=train_x/255.0
test_x=test_x/255.0

In [9]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [11]:
training_set = train_datagen.flow_from_directory(path_train,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'sparse')
test_set = test_datagen.flow_from_directory(path_test,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'sparse')

Found 200 images belonging to 2 classes.
Found 95 images belonging to 2 classes.


In [12]:
train_y=training_set.classes
test_y=test_set.classes

In [13]:
training_set.class_indices

{'false': 0, 'true': 1}

In [14]:
IMAGE_SIZE = [224, 224]
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

80150528/80134624 [==============================] - 0s 0us/step


In [18]:
from keras.applications.vgg19 import VGG19
vgg = VGG19(weights='imagenet')

for layer in vgg.layers:
    layer.trainable = False
x = Flatten()(vgg.output)
prediction = Dense(2, activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)


In [19]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [24]:
model.compile(
  loss = 'sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

In [25]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)

In [30]:
history = model.fit(
  np.array(train_x),
  np.array(train_y),
  epochs=10,
  callbacks=[early_stop],
  batch_size=32,
  shuffle=True
  )

Epoch 1/10
7/7 [==============================] - ETA: 0s - loss: 0.6933 - accuracy: 0.5000 

7/7 [==============================] - 138s 19s/step - loss: 0.6933 - accuracy: 0.5000
Epoch 2/10
7/7 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5000 

7/7 [==============================] - 129s 18s/step - loss: 0.6931 - accuracy: 0.5000
Epoch 3/10
7/7 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.5000 

7/7 [==============================] - 133s 19s/step - loss: 0.6932 - accuracy: 0.5000
Epoch 4/10
7/7 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5100 

7/7 [==============================] - 132s 18s/step - loss: 0.6931 - accuracy: 0.5100
Epoch 5/10
7/7 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.5100 

...

7/7 [==============================] - 129s 18s/step - loss: 0.6931 - accuracy: 0.5000
Epoch 7/10
7/7 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5000 

7/7 [==============================] - 132s 18s/step - loss: 0.6931 - accuracy: 0.5000
Epoch 8/10
7/7 [==============================] - ETA: 0s - loss: 0.6934 - accuracy: 0.5000 

7/7 [==============================] - 136s 19s/step - loss: 0.6934 - accuracy: 0.5000
Epoch 9/10
7/7 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5000 

7/7 [==============================] - 129s 18s/step - loss: 0.6931 - accuracy: 0.5000
Epoch 10/10
7/7 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.5000 

7/7 [==============================] - 132s 19s/step - loss: 0.6932 - accuracy: 0.5000
